In [1]:
# 모델 학습 코드
import os
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainerCallback
from datasets import Dataset
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig

[2025-02-16 14:20:13,005] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [2]:
# 모델 및 데이터 설정
base_model_id = "dnotitia/Llama-DNA-1.0-8B-Instruct"
device_map = "cuda"
torch_dtype = torch.bfloat16
output_dir = "output"
dataset_name = "data/augmented_train_output.csv"

In [3]:
# 최대 입력/출력 길이 설정 
seq_length = 3000

In [4]:
# CSV 데이터 로드
try:
    full_dataset = Dataset.from_csv(path_or_paths=dataset_name)
    print("CSV 파일이 성공적으로 로드되었습니다.")
except Exception as e:
    print(f"CSV 파일 로드 중 오류 발생: {e}")
    raise

CSV 파일이 성공적으로 로드되었습니다.


In [5]:
# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, legacy=True)
tokenizer.padding_side = "right"
tokenizer.truncation = True  
tokenizer.max_length = seq_length

Exception: data did not match any variant of untagged enum ModelWrapper at line 1251003 column 3

In [ ]:
# LoRA 구성
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "down_proj", "up_proj", "gate_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
# 4-bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
)

# 캐시 비활성화
base_model.config.use_cache = False

In [ ]:
# 토크나이저 패딩 토큰 설정
if getattr(tokenizer, "pad_token", None) is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
# 입력 및 출력 포맷 준비 함수 
def function_prepare_sample_text(tokenizer, for_train=True):
    def _prepare_sample_text(example):
        system_prompt = (
            "당신은 한국어 리뷰 복원 전문가입니다.\n"
            "당신의 임무는 난독화된 한글 리뷰를 분석하고, 이를 자연스럽고 명확한 원래 의미의 한글 리뷰로 복원하는 것입니다.\n"
            "난독화된 리뷰의 단어를 원본 단어로 복원하고, 띄어쓰기와 문장 구조도 원래대로 복원하세요.\n"
            "문맥을 분석하여 자연스럽고 의미 있는 복원을 수행하며, 출력은 오직 한국어로만 작성하십시오."
        )
        
        user_prompt = example.get("input", "")
        if not isinstance(user_prompt, str):
            user_prompt = "" 

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        if for_train:
            assistant_response = example.get("output", "")
            if not isinstance(assistant_response, str):
                assistant_response = ""

            messages.append({"role": "assistant", "content": assistant_response})

        # 메시지를 문자열로 변환 (apply_chat_template가 항상 str을 반환해야 함)
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=not for_train)
        
        if not isinstance(text, str):
            raise ValueError(f"apply_chat_template 결과가 문자열이 아닙니다. text={text}")

        return text
    return _prepare_sample_text

In [ ]:
# 샘플 텍스트 준비 및 토큰 길이 대비 문자 비율 계산
def chars_token_ratio(dataset, tokenizer, prepare_sample_text, nb_examples=400):
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = prepare_sample_text(example)
        total_characters += len(text)
        total_tokens += len(tokenizer(text).tokens())
    return total_characters / total_tokens

In [ ]:
# 데이터셋 생성
def create_datasets(tokenizer, dataset, seq_length):
    prepare_sample_text = function_prepare_sample_text(tokenizer)
    chars_per_token = chars_token_ratio(dataset, tokenizer, prepare_sample_text)
    print(f"문자 대비 토큰 비율: {chars_per_token:.2f}")
    
    cl_dataset = ConstantLengthDataset(
        tokenizer,
        dataset,
        formatting_func=prepare_sample_text,
        infinite=True,
        seq_length=seq_length,
        chars_per_token=chars_per_token,
    )
    return cl_dataset

ds = create_datasets(tokenizer, full_dataset, seq_length)

In [ ]:
# SFT 설정 
sft_config = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=2,  
    gradient_accumulation_steps=3, 
    gradient_checkpointing=True,
    learning_rate=5e-5,
    warmup_ratio=0.1,
    max_grad_norm=0.3,
    weight_decay=0.05,
    num_train_epochs=3,  
    logging_steps=500,
    eval_strategy="no", 
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=3,
    max_seq_length=seq_length, 
    report_to="wandb",
    run_name="dacon_finetuning_2_2"
)

In [ ]:
# 트레이너 설정 및 학습 시작
trainer = SFTTrainer(
    model=base_model,
    train_dataset=ds,
    eval_dataset=None,
    peft_config=lora_config,  # LoRA 적용
    tokenizer=tokenizer,
    args=sft_config,
    formatting_func=function_prepare_sample_text(tokenizer, for_train=True)
)

trainer.train()